# 05 - Testes de Hipótese

Neste notebook, são realizados testes de hipótese voltados à comparação das acurácias.

 As acurácias a serem comparadas advém de:
+ Máquinas treinadas sem ensemble 
+ Máquinas treinadas com ensemble não balanceado
+ Máquinas treinadas com ensemble balanceado com Undersampling aleatório
+ Máquinas treinadas com ensemble balanceado com Oversampling - SMOTE

## Importação

In [1]:
# Bibliotecas padrão
import pickle

# Bibliotecas utilitárias de terceiros
import numpy as np
import pandas as pd
import scipy.stats as ss
from tqdm.notebook import tqdm

# Gráficos
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

## Funções

In [2]:
def plot_accuracies(res):
    # # Criando uma lista de traces para cada categoria
    # traces = []
    # for categoria, valores in res.items():
    #     traces.append(go.Histogram(x=valores, name=categoria, opacity=0.6))

    # # Criando o gráfico
    # fig = go.Figure(data=traces)

    # # Configurando a sobreposição de histogramas
    # fig.update_layout(title=f'Distribuição das Acurácias por abordagem',
    #                   yaxis_title='Frequência',
    #                   xaxis_title='Acurácia (%)',
    #                   height=600, autosize=True,
    #                   barmode='overlay',
    #                   legend=dict(
    #                       orientation="h", 
    #                       yanchor="bottom", 
    #                       y=1.02, 
    #                       xanchor="right", 
    #                       x=1
    #                       )
    #                   )
    
    fig = make_subplots(rows=len(res), cols=1, shared_xaxes=True, shared_yaxes=True, subplot_titles=list(res.keys()))

    i=1
    for categoria, valores in res.items():
        fig.add_trace(
            go.Histogram(x=valores, name=categoria),
            row=i,
            col=1
        )
        i=i+1

    fig.update_layout(title=f'Distribuição das Acurácias por abordagem',
                      yaxis1_title='Frequência',
                      yaxis2_title='Frequência',
                      yaxis3_title='Frequência',
                      yaxis4_title='Frequência',
                      xaxis4_title='Acurácia (%)',
                      height=800, autosize=True,
                      barmode='overlay',
                      yaxis1_range=(0, 10),
                      yaxis2_range=(0, 10),
                      yaxis3_range=(0, 10),
                      yaxis4_range=(0, 10),
                      showlegend=False
                    #   legend=dict(
                    #       orientation="h", 
                    #       yanchor="bottom", 
                    #       y=1.02, 
                    #       xanchor="right", 
                    #       x=1
                    #       )
                      )

    # Exibindo o gráfico
    fig.show()

## KNN

In [3]:
with open('../data/processed/knn_res.pkl', 'rb') as file: 
    knn_res = pickle.load(file)

with open('../data/processed/knn_imb_bag_res.pkl', 'rb') as file: 
    knn_imb_bag_res = pickle.load(file)

with open('../data/processed/knn_bal_und_bag_res.pkl', 'rb') as file: 
    knn_bal_und_bag_res = pickle.load(file)

with open('../data/processed/knn_bal_ovr_bag_res.pkl', 'rb') as file: 
    knn_bal_ovr_bag_res = pickle.load(file)

knn_accs = {
    'Sem bagging': knn_res['accuracies'],
    'Com bagging sem balanceamento': knn_imb_bag_res['accuracies'],
    'Com bagging com balanceamento undersampling aleatório': knn_bal_und_bag_res['accuracies'],
    'Com bagging com balanceamento oversampling': knn_bal_ovr_bag_res['accuracies'],
}

plot_accuracies(knn_accs)

### Testes das premissas para escolha adequada do teste

#### Teste de normalidade

Para a escolha do teste que será aplicado, faz-se necessário primeiramente a verificação da normalidade das amostras. Dito isto, vamos realizar o teste de normalidade nos nossos dados. Adotaremos $\alpha = 5\%$ para o nosso teste, cujas hipóteses são:
* **H0:** as amostras vêm de uma população com distribuição normal
* **H1:** as amostras não vêm de uma população com distribuição normal

In [4]:
for strategy, strategy_accs in knn_accs.items():
     norm_test = ss.normaltest(strategy_accs)
     p_values = norm_test.pvalue

     print(f'{strategy}: {p_values}')

Sem bagging: 0.38764332083486225
Com bagging sem balanceamento: 0.6765025876702073
Com bagging com balanceamento undersampling aleatório: 0.6726101313070068
Com bagging com balanceamento oversampling: 0.942241954457596


A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que elas seguem uma distribuição normal.

#### Teste de variância

Não descartada a hipótese que os dados seguem uma distribuição normal, verifiquemos agora a hipótese de que as amostras advém de populações com variâncias iguais. Dito isto, vamos realizar testes para esta verificação. Adotaremos $\alpha = 5\%$ para o nosso teste, cujas hipóteses são:
* **H0:** as amostras vêm de uma população com variância igual.
* **H1:** as amostras vêm de uma população com variância diferente.

In [5]:
# O teste de Levene é robusto contra desvios da normalidade.
ss.levene(
    knn_accs['Sem bagging'],
    knn_accs['Com bagging sem balanceamento'],
    knn_accs['Com bagging com balanceamento undersampling aleatório'],
    knn_accs['Com bagging com balanceamento oversampling']
)

LeveneResult(statistic=np.float64(2.4337591615390712), pvalue=np.float64(0.07136465503782843))

A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que as variâncias (e, portanto, os desvios-padrão) são consideradas iguais.

In [6]:
# O teste de Bartlett é mais sensível a desvios da normalidade.
ss.bartlett(
    knn_accs['Sem bagging'],
    knn_accs['Com bagging sem balanceamento'],
    knn_accs['Com bagging com balanceamento undersampling aleatório'],
    knn_accs['Com bagging com balanceamento oversampling']
)

BartlettResult(statistic=np.float64(6.504859906697977), pvalue=np.float64(0.08947103819892134))

A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que as variâncias (e, portanto, os desvios-padrão) são consideradas iguais.

### Testes de Hipótese sobre a Média

#### Teste T para 2 Amostras

In [7]:
ss.ttest_ind(knn_accs['Sem bagging'], knn_accs['Com bagging sem balanceamento'], alternative="less", equal_var=True)

TtestResult(statistic=np.float64(-0.6410853798835312), pvalue=np.float64(0.26265753633737876), df=np.float64(38.0))

In [8]:
ss.ttest_ind(knn_accs['Sem bagging'], knn_accs['Com bagging sem balanceamento'], alternative="less", equal_var=False)

TtestResult(statistic=np.float64(-0.6410853798835313), pvalue=np.float64(0.2627760416569409), df=np.float64(35.78441069723809))

In [9]:
ss.mannwhitneyu(knn_accs['Sem bagging'], knn_accs['Com bagging sem balanceamento'], alternative="less")

MannwhitneyuResult(statistic=np.float64(183.0), pvalue=np.float64(0.3266047935247083))

#### ANOVA

Hipóteses
* **H0:** os grupos possuem a mesma média populacional
* **H1:** os grupos possuem média populacional diferente

In [10]:
ss.f_oneway(
    knn_accs['Sem bagging'],
    knn_accs['Com bagging sem balanceamento'],
    knn_accs['Com bagging com balanceamento undersampling aleatório'],
    knn_accs['Com bagging com balanceamento oversampling']
)

F_onewayResult(statistic=np.float64(11.744627952837524), pvalue=np.float64(2.100959374850116e-06))

A 5% de significância há evidências estatísticas suficientes para rejeitar a hipótese nula de que os grupos possuam média populacional igual.

#### Kruskal-Wallis

Hipóteses
* **H0:** os grupos possuem a mesma média populacional
* **H1:** os grupos possuem média populacional diferente

In [11]:
ss.kruskal(
    knn_accs['Sem bagging'],
    knn_accs['Com bagging sem balanceamento'],
    #knn_accs['Com bagging com balanceamento undersampling aleatório'],
    knn_accs['Com bagging com balanceamento oversampling']
)

KruskalResult(statistic=np.float64(0.7878610209336043), pvalue=np.float64(0.674400918417219))

A 5% de significância há evidências estatísticas suficientes para rejeitar a hipótese nula de que os grupos possuam média populacional igual.

## DT

In [12]:
with open('../data/processed/dt_res.pkl', 'rb') as file: 
    dt_res = pickle.load(file)

with open('../data/processed/dt_imb_bag_res.pkl', 'rb') as file: 
    dt_imb_bag_res = pickle.load(file)

with open('../data/processed/dt_bal_und_bag_res.pkl', 'rb') as file: 
    dt_bal_und_bag_res = pickle.load(file)

with open('../data/processed/dt_bal_ovr_bag_res.pkl', 'rb') as file: 
    dt_bal_ovr_bag_res = pickle.load(file)

dt_accs = {
    'Sem bagging': dt_res['accuracies'],
    'Com bagging sem balanceamento': dt_imb_bag_res['accuracies'],
    'Com bagging com balanceamento undersampling aleatório': dt_bal_und_bag_res['accuracies'],
    'Com bagging com balanceamento oversampling': dt_bal_ovr_bag_res['accuracies'],
}

plot_accuracies(dt_accs)

### Testes das premissas para escolha adequada do teste

#### Teste de normalidade

Para a escolha do teste que será aplicado, faz-se necessário primeiramente a verificação da normalidade das amostras. Dito isto, vamos realizar o teste de normalidade nos nossos dados. Adotaremos $\alpha = 5\%$ para o nosso teste, cujas hipóteses são:
* **H0:** as amostras vêm de uma população com distribuição normal
* **H1:** as amostras não vêm de uma população com distribuição normal

In [13]:
for strategy, strategy_accs in dt_accs.items():
     norm_test = ss.normaltest(strategy_accs)
     p_values = norm_test.pvalue

     print(f'{strategy}: {p_values}')

Sem bagging: 0.6823828866431272
Com bagging sem balanceamento: 0.19873564644806507
Com bagging com balanceamento undersampling aleatório: 0.3350656470606593
Com bagging com balanceamento oversampling: 0.706813037974168


A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que elas seguem uma distribuição normal.

#### Teste de variância

Não descartada a hipótese que os dados seguem uma distribuição normal, verifiquemos agora a hipótese de que as amostras advém de populações com variâncias iguais. Dito isto, vamos realizar testes para esta verificação. Adotaremos $\alpha = 5\%$ para o nosso teste, cujas hipóteses são:
* **H0:** as amostras vêm de uma população com variância igual.
* **H1:** as amostras vêm de uma população com variância diferente.

In [14]:
# O teste de Levene é robusto contra desvios da normalidade.
ss.levene(
    dt_accs['Sem bagging'],
    dt_accs['Com bagging sem balanceamento'],
    dt_accs['Com bagging com balanceamento undersampling aleatório'],
    dt_accs['Com bagging com balanceamento oversampling']
)

LeveneResult(statistic=np.float64(0.5310702609143292), pvalue=np.float64(0.6623319823410789))

A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que as variâncias (e, portanto, os desvios-padrão) são consideradas iguais.

In [15]:
# O teste de Bartlett é mais sensível a desvios da normalidade.
ss.bartlett(
    dt_accs['Sem bagging'],
    dt_accs['Com bagging sem balanceamento'],
    dt_accs['Com bagging com balanceamento undersampling aleatório'],
    dt_accs['Com bagging com balanceamento oversampling']
)

BartlettResult(statistic=np.float64(1.407182388832778), pvalue=np.float64(0.7038520062964166))

A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que as variâncias (e, portanto, os desvios-padrão) são consideradas iguais.

### Testes de Hipótese sobre a Média

#### Teste T para 2 Amostras

In [16]:
ss.ttest_ind(dt_accs['Sem bagging'], dt_accs['Com bagging com balanceamento oversampling'], alternative="less", equal_var=True)

TtestResult(statistic=np.float64(-3.3359758471062566), pvalue=np.float64(0.0009541188969529036), df=np.float64(38.0))

In [17]:
ss.ttest_ind(dt_accs['Sem bagging'], dt_accs['Com bagging com balanceamento oversampling'], alternative="less", equal_var=False)

TtestResult(statistic=np.float64(-3.3359758471062566), pvalue=np.float64(0.0009867325636060448), df=np.float64(36.21217149774423))

In [18]:
ss.mannwhitneyu(dt_accs['Sem bagging'], dt_accs['Com bagging com balanceamento oversampling'], alternative="less")

MannwhitneyuResult(statistic=np.float64(90.5), pvalue=np.float64(0.0015641462718274956))

#### ANOVA

Hipóteses
* **H0:** os grupos possuem a mesma média populacional
* **H1:** os grupos possuem média populacional diferente

In [19]:
ss.f_oneway(
    dt_accs['Sem bagging'],
    dt_accs['Com bagging sem balanceamento'],
    dt_accs['Com bagging com balanceamento undersampling aleatório'],
    dt_accs['Com bagging com balanceamento oversampling']
)

F_onewayResult(statistic=np.float64(4.460223677162361), pvalue=np.float64(0.006118339323982459))

A 5% de significância há evidências estatísticas suficientes para rejeitar a hipótese nula de que os grupos possuam média populacional igual.

#### Kruskal-Wallis

Hipóteses
* **H0:** os grupos possuem a mesma média populacional
* **H1:** os grupos possuem média populacional diferente

In [20]:
ss.kruskal(
    dt_accs['Sem bagging'],
    dt_accs['Com bagging sem balanceamento'],
    dt_accs['Com bagging com balanceamento undersampling aleatório'],
    dt_accs['Com bagging com balanceamento oversampling']
)

KruskalResult(statistic=np.float64(10.522752782549395), pvalue=np.float64(0.014607343591262308))

A 5% de significância há evidências estatísticas suficientes para rejeitar a hipótese nula de que os grupos possuam média populacional igual.

## SVM

In [21]:
with open('../data/processed/svc_res.pkl', 'rb') as file: 
    svm_res = pickle.load(file)

with open('../data/processed/svm_imb_bag_res.pkl', 'rb') as file: 
    svm_imb_bag_res = pickle.load(file)

with open('../data/processed/svm_bal_und_bag_res.pkl', 'rb') as file: 
    svm_bal_und_bag_res = pickle.load(file)

with open('../data/processed/svm_bal_ovr_bag_res.pkl', 'rb') as file: 
    svm_bal_ovr_bag_res = pickle.load(file)

svm_accs = {
    'Sem bagging': svm_res['accuracies'],
    'Com bagging sem balanceamento': svm_imb_bag_res['accuracies'],
    'Com bagging com balanceamento undersampling aleatório': svm_bal_und_bag_res['accuracies'],
    'Com bagging com balanceamento oversampling': svm_bal_ovr_bag_res['accuracies'],
}

plot_accuracies(svm_accs)

### Testes das premissas para escolha adequada do teste

#### Teste de normalidade

Para a escolha do teste que será aplicado, faz-se necessário primeiramente a verificação da normalidade das amostras. Dito isto, vamos realizar o teste de normalidade nos nossos dados. Adotaremos $\alpha = 5\%$ para o nosso teste, cujas hipóteses são:
* **H0:** as amostras vêm de uma população com distribuição normal
* **H1:** as amostras não vêm de uma população com distribuição normal

In [22]:
for strategy, strategy_accs in svm_accs.items():
     norm_test = ss.normaltest(strategy_accs)
     p_values = norm_test.pvalue

     print(f'{strategy}: {p_values}')

Sem bagging: 0.34668409040691095
Com bagging sem balanceamento: 0.8626957897516464
Com bagging com balanceamento undersampling aleatório: 0.3346715101795784
Com bagging com balanceamento oversampling: 0.9886139212569551


A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que elas seguem uma distribuição normal.

#### Teste de variância

Não descartada a hipótese que os dados seguem uma distribuição normal, verifiquemos agora a hipótese de que as amostras advém de populações com variâncias iguais. Dito isto, vamos realizar testes para esta verificação. Adotaremos $\alpha = 5\%$ para o nosso teste, cujas hipóteses são:
* **H0:** as amostras vêm de uma população com variância igual.
* **H1:** as amostras vêm de uma população com variância diferente.

In [23]:
# O teste de Levene é robusto contra desvios da normalidade.
ss.levene(
    svm_accs['Sem bagging'],
    svm_accs['Com bagging sem balanceamento'],
    svm_accs['Com bagging com balanceamento undersampling aleatório'],
    svm_accs['Com bagging com balanceamento oversampling']
)

LeveneResult(statistic=np.float64(2.0845579399890193), pvalue=np.float64(0.10926285401162775))

A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que as variâncias (e, portanto, os desvios-padrão) são consideradas iguais.

In [24]:
# O teste de Bartlett é mais sensível a desvios da normalidade.
ss.bartlett(
    svm_accs['Sem bagging'],
    svm_accs['Com bagging sem balanceamento'],
    svm_accs['Com bagging com balanceamento undersampling aleatório'],
    svm_accs['Com bagging com balanceamento oversampling']
)

BartlettResult(statistic=np.float64(4.495126810780289), pvalue=np.float64(0.2127253768152496))

A 5% de significância não há evidências estatísticas suficientes para rejeitar a hipótese nula de que as variâncias (e, portanto, os desvios-padrão) são consideradas iguais.

### Testes de Hipótese sobre a Média

#### Teste T para 2 Amostras

In [25]:
ss.ttest_ind(svm_accs['Sem bagging'], svm_accs['Com bagging com balanceamento oversampling'], alternative="less", equal_var=True)

TtestResult(statistic=np.float64(-0.06862213556531029), pvalue=np.float64(0.47282511395318527), df=np.float64(38.0))

In [26]:
ss.ttest_ind(svm_accs['Sem bagging'], svm_accs['Com bagging com balanceamento oversampling'], alternative="less", equal_var=False)

TtestResult(statistic=np.float64(-0.06862213556531029), pvalue=np.float64(0.4728316168044153), df=np.float64(36.669740598915794))

In [27]:
ss.mannwhitneyu(svm_accs['Sem bagging'], svm_accs['Com bagging com balanceamento oversampling'], alternative="less")

MannwhitneyuResult(statistic=np.float64(193.5), pvalue=np.float64(0.4352145205383054))

#### ANOVA

Hipóteses
* **H0:** os grupos possuem a mesma média populacional
* **H1:** os grupos possuem média populacional diferente

In [28]:
ss.f_oneway(
    svm_accs['Sem bagging'],
    svm_accs['Com bagging sem balanceamento'],
    svm_accs['Com bagging com balanceamento undersampling aleatório'],
    svm_accs['Com bagging com balanceamento oversampling']
)

F_onewayResult(statistic=np.float64(5.017643101163769), pvalue=np.float64(0.0031573138012610117))

A 5% de significância há evidências estatísticas suficientes para rejeitar a hipótese nula de que os grupos possuam média populacional igual.

#### Kruskal-Wallis

Hipóteses
* **H0:** os grupos possuem a mesma média populacional
* **H1:** os grupos possuem média populacional diferente

In [29]:
ss.kruskal(
    svm_accs['Sem bagging'],
    svm_accs['Com bagging sem balanceamento'],
    svm_accs['Com bagging com balanceamento undersampling aleatório'],
    svm_accs['Com bagging com balanceamento oversampling']
)

KruskalResult(statistic=np.float64(10.296751480899102), pvalue=np.float64(0.01620497564848509))

A 5% de significância há evidências estatísticas suficientes para rejeitar a hipótese nula de que os grupos possuam média populacional igual.